# Install Packages

In [52]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString
import matplotlib.pyplot as plt
import json
from shapely.validation import make_valid
import warnings

# Read Files

In [ ]:
data = pd.read_json("data/cleaned/BusRoutes.json", encoding="utf-8", lines=True, chunksize=1000)

# Combine chunks into a dataframe
df_list = []
for chunk in data:
    df_list.append(chunk)

# Concatenate all chunks into a single dataframe
bus_routes = pd.concat(df_list, ignore_index=True)
bus_routes = bus_routes[bus_routes['Direction']==1]
bus_routes = bus_routes[['ServiceNo', 'BusStopCode', 'StopSequence', 'Distance']]
print(bus_routes.head())

In [ ]:
# Load BusStops.geojson using GeoPandas
bus_stops = gpd.read_file('data/cleaned/BusStops.geojson').to_crs(32648)
print(bus_stops.head())

In [ ]:
rail_line = gpd.read_file('data/cleaned/RailLines.geojson').to_crs(32648)
print(rail_line.head())

In [ ]:
rail_stn = gpd.read_file('data/cleaned/RailStationsMerged.geojson').to_crs(32648)
print(rail_stn.head())

# Data Cleaning : Obtain  Bus Line Strings

In [ ]:
## Merge Bus Stop Coordinates with Bus Routes
# Ensure that BusStopCode is of the same type for both DataFrames (string)
bus_stops['BusStopCode'] = bus_stops['BUS_STOP_N'].astype(str)
bus_routes['BusStopCode'] = bus_routes['BusStopCode'].astype(str)

# Merge bus_routes with bus_stops to get geometry for each bus stop
merged_bus_data = bus_routes.merge(bus_stops[['BusStopCode', 'geometry']], on='BusStopCode', how='left')

# Drop rows without geometry
merged_bus_data = merged_bus_data.dropna(subset=['geometry'])

# Convert to GeoDataFrame
bus_routes_geo = gpd.GeoDataFrame(merged_bus_data, geometry='geometry')
# bus_routes_geo.to_crs(3857)
bus_routes_geo.set_crs(epsg=32648, inplace=True,allow_override=True)
bus_routes_geo.head()

# Segment Bus Routes by LineString


In [ ]:
bus_routes_lines = (
    bus_routes_geo.groupby(['ServiceNo'])
    .apply(lambda x: LineString(x.sort_values('StopSequence')['geometry'].tolist()))
    .reset_index()
    .rename(columns={0: 'geometry'})
)
# Convert to GeoDataFrame
bus_routes_lines = gpd.GeoDataFrame(bus_routes_lines, geometry='geometry')
bus_routes_lines = bus_routes_lines.set_crs(epsg=32648)
print(bus_routes_lines.head())

bs10 = bus_routes_lines[bus_routes_lines['ServiceNo']=='10']

bs10.plot()
plt.show()

# Visualise Bus Segments

In [ ]:
def segment_line_by_distance(line, distance=1000):
    segments = []  # List to store the line segments
    current_distance = 0  # Start distance

    # Iterate over the entire length of the line and create segments of the specified distance
    while current_distance < line.length:
        # Calculate the start and end points for each segment
        start_point = line.interpolate(current_distance)
        end_point = line.interpolate(min(current_distance + distance, line.length))
        
        # Create a LineString for the segment and add it to the list
        segments.append(LineString([start_point, end_point]))
        
        # Move to the next segment
        current_distance += distance

    return segments  # Return the list of 1 km line segments

# Segment the line by 1 km distance
bs10_segmented = segment_line_by_distance(bs10.geometry.iloc[0], 1000)
print(bs10_segmented)
# plot orignal bus line
x, y = bs10.geometry.iloc[0].xy  # Extract x and y coordinates for the original line
plt.plot(x, y, color='gray', linewidth=3, label='Original Bus Line')

# plot segmented bus line
for segment in bs10_segmented:
    x_seg, y_seg = segment.xy  # Extract x and y coordinates for the segment
    plt.plot(x_seg, y_seg, linewidth=2, linestyle='--')

# Show the plot
plt.grid(True)
plt.show()

# Create MRT Line String with MRT Stations

In [ ]:
rail_stn = rail_stn.to_crs(epsg=32648)

#  Group the stations by MRT line
rail_stn.drop(rail_stn[rail_stn['StationCode']=='PTC'].index, inplace=True)
rail_stn.drop(rail_stn[rail_stn['StationCode']=='STC'].index, inplace=True)
rail_stn.replace("TE22A", "TE22.5", inplace=True)
station_groups = rail_stn.groupby('StationLine')

# Dictionary to store LineStrings for each MRT line
line_strings_per_station_line = {}

# Iterate over each MRT line group and create a sorted LineString
for line_name, group in station_groups:
    # Sort the stations by their longitude (x-coordinate) or latitude (y-coordinate)
    group['centroid_x'] = group.geometry.centroid.x  # Get x-coordinate of the centroid
    group['StationCodeNo'] = group['StationCode'].str[2:].astype(float)
    sorted_group = group.sort_values('StationCodeNo')  # Sort by the x-coordinate

    # Extract the station centroids, names, and codes
    station_points = sorted_group.geometry.centroid.tolist()
    station_names = sorted_group['StationName'].tolist()
    station_codes = sorted_group['StationCode'].tolist()

    # Initialize a set to track unique station codes
    unique_station_codes = set()

    # Initialize lists to store filtered stations (with unique codes)
    filtered_station_points = []
    filtered_station_names = []
    filtered_station_codes = []

    #Filter out stations with duplicate station codes and None values
    for point, name, code in zip(station_points, station_names, station_codes):
        if code not in unique_station_codes and point is not None:
            # Add the station to the filtered lists
            filtered_station_points.append(point)
            filtered_station_names.append(name)
            filtered_station_codes.append(code)

            # Mark this station code as seen
            unique_station_codes.add(code)

    # create a LineString from the filtered station points
    if filtered_station_points:  # Check if we have valid points
        try:
            mrt_line = LineString(filtered_station_points)
        
            # Store the data in the dictionary
            line_strings_per_station_line[line_name] = {
                'LineString': mrt_line,
                'StationNames': filtered_station_names,
                'StationCodes': filtered_station_codes,
                'StationPoints': filtered_station_points
            }
        except TypeError as e:
            print(f"Error creating LineString for {line_name}: {e}")
            continue

print(line_strings_per_station_line)
for line_name, line_data in line_strings_per_station_line.items():
    print(f"MRT Line: {line_name}")
    print(f"  LineString: {line_data['LineString']}")
    print(f"  Station Names: {line_data['StationNames']}")
    print(f"  Station Codes: {line_data['StationCodes']}")
    print(f"  Station Points (centroids): {line_data['StationPoints']}")
    print("\n")

# Visualise MRT Line String

In [ ]:

target_line = "Circle" #change accordingly

# Check if the line exists in the processed data
if target_line in line_strings_per_station_line:
    line_data = line_strings_per_station_line[target_line]

    # Extract the LineString, station points, and station names
    mrt_line = line_data['LineString']
    station_points = line_data['StationPoints']
    station_names = line_data['StationNames']

    # Plot the MRT line
    plt.figure(figsize=(10, 8))
    
    # Plot the LineString representing the MRT line
    x, y = mrt_line.xy  # Extract x and y coordinates from the LineString
    plt.plot(x, y, label=f'{target_line} Line', color='blue', linewidth=2)
    
    # Plot the station points on the map
    station_x = [point.x for point in station_points]
    station_y = [point.y for point in station_points]
    plt.scatter(station_x, station_y, color='red', label='Stations', zorder=5)

    # Annotate the station names
    for i, name in enumerate(station_names):
        plt.text(station_x[i], station_y[i], name, fontsize=8, ha='right')

    # Set the title and labels
    plt.title(f'{target_line} MRT Line with Stations')
    plt.xlabel('Longitude (meters)')
    plt.ylabel('Latitude (meters)')
    
    # Add a legend
    plt.legend()
    
    # Show the plot
    plt.grid(True)
    plt.show()

else:
    print(f"No data available for MRT Line: {target_line}")


# Segment MRT Stations

In [34]:
# Function to segment a LineString into 1 km sections
def segment_line_by_distance(line, distance=1000):
    segments = []  # List to store the line segments
    current_distance = 0  # Start distance

    # Iterate over the entire length of the line and create segments of the specified distance
    while current_distance < line.length:
        # Calculate the start and end points for each segment
        start_point = line.interpolate(current_distance)
        end_point = line.interpolate(min(current_distance + distance, line.length))
        
        # Create a LineString for the segment and add it to the list
        segments.append(LineString([start_point, end_point]))
        
        # Move to the next segment
        current_distance += distance

    return segments  # Return the list of 1 km line segments

# Dictionary to store segmented lines for each MRT line
segmented_lines_per_station_line = {}

# Iterate through each MRT line in the dataset
for line_name, line_info in line_strings_per_station_line.items():
    # Get the LineString and station details for each line
    mrt_line = line_info['LineString']
    station_points = line_info['StationPoints']
    station_names = line_info['StationNames']

    # Segment the MRT line into 1 km sections
    mrt_segments = segment_line_by_distance(mrt_line, 1000)

    # Store the segments for this MRT line
    segmented_lines_per_station_line[line_name] = {
        'segments': mrt_segments,
        'station_names': station_names,
        'station_points': station_points
    }


# Visualise MRT Segments

In [ ]:
target_line = "Circle"  # change to visualize other lines
if target_line in segmented_lines_per_station_line:
    segmented_data = segmented_lines_per_station_line[target_line]

    mrt_segments = segmented_data['segments']
    station_points = segmented_data['station_points']
    station_names = segmented_data['station_names']

    # Visualize the segments on the map
    plt.figure(figsize=(10, 8))

    # Plot each segment in a different color
    for idx, segment in enumerate(mrt_segments):
        x_seg, y_seg = segment.xy  # Extract x and y coordinates for the segment
        plt.plot(x_seg, y_seg, label=f'Segment {idx+1}', linewidth=2)

    # Plot the station points
    station_x = [point.x for point in station_points]
    station_y = [point.y for point in station_points]
    plt.scatter(station_x, station_y, color='red', label='Stations', zorder=5)

    # Annotate the station names
    for i, name in enumerate(station_names):
        plt.text(station_x[i], station_y[i], name, fontsize=8, ha='right')

    # Set the title and labels
    plt.title(f'{target_line} MRT Line Segmented into 1 km Sections')
    plt.xlabel('Longitude (meters)')
    plt.ylabel('Latitude (meters)')

    # Add a legend
    plt.legend()
    
    # Show the plot
    plt.grid(True)
    plt.show()

    # Print the first few segments for verification
    for idx, segment in enumerate(mrt_segments[:5]):  # Print first 5 segments
        print(f"Segment {idx+1}: {segment}")

else:
    print(f"No data available for MRT Line: {target_line}")

# PHASE 1: Conduct Initial Checks for Distance Proximity
Compare the bus route's entire geometry with the MRT line to check if they are near in distance. If there's no overlap, discard that bus route.
Reduces the number of comparisons needed.

In [36]:
# Define buffer distance
buffer_distance = 500

In [ ]:
if bus_routes_lines.crs is None:
    bus_routes_lines = bus_routes_lines.set_crs(epsg=4326)

#Reproject bus routes to the same CRS as the MRT lines (UTM Zone 48N - EPSG:32648)
bus_routes_lines = bus_routes_lines.to_crs(epsg=32648)

mrt_buffers = {}  # Dictionary to store buffers for each MRT line

for line_name, line_data in line_strings_per_station_line.items():
    mrt_line = line_data['LineString']
    mrt_buffer = mrt_line.buffer(buffer_distance)  # Create a buffer around the MRT line
    mrt_buffers[line_name] = mrt_buffer  # Store the buffer

#Check if any bus route is near any MRT line buffer
overlap_results = []

for line_name, mrt_buffer in mrt_buffers.items():
    for idx, bus_route in bus_routes_lines.iterrows():
        if bus_route.geometry.intersects(mrt_buffer):
            overlap_length = bus_route.geometry.intersection(mrt_buffer).length  # Get overlap length
            overlap_results.append({
                'MRT_Line': line_name,
                'Bus_ServiceNo': bus_route['ServiceNo'],
                'Direction': bus_route['Direction'],  # Keep track of direction for later
                'Bus_Route_Length_m': bus_route.geometry.length,  # Length of the bus route in meters
                'Overlap_Length_m': overlap_length  # Overlap length in meters
            })

overlap_df = pd.DataFrame(overlap_results)

#Sort the DataFrame by 'Overlap_Length_m' in descending order
overlap_df_sorted = overlap_df.sort_values(by='Overlap_Length_m', ascending=False)

# For each bus service, keep only the direction with the greater overlap
# Group by 'Bus_ServiceNo' and 'Direction', take the max 'Overlap_Length_m' for each service
overlap_df_max = overlap_df_sorted.groupby(['Bus_ServiceNo', 'Direction']).apply(lambda x: x.loc[x['Overlap_Length_m'].idxmax()]).reset_index(drop=True)

# Drop duplicates for each 'Bus_ServiceNo' while keeping only the row with the maximum 'Overlap_Length_m'
overlap_df_unique = overlap_df_max.sort_values(by='Overlap_Length_m', ascending=False).drop_duplicates(subset=['Bus_ServiceNo'], keep='first')

# Output the final DataFrame
overlap_df_unique



# PHASE 2.1: Segment Coverage Calculation for Bus Routes and MRT Line Segments
To determine how much of an MRT line's segments are covered by a bus route. The MRT lines are segmented into equal parts, and we check which of these segments overlap with the corresponding bus route. This analysis helps us assess how parallel the bus route is to the MRT line in terms of spatial coverage.

Steps:

Segmentation: Each MRT line is divided into segments of a fixed length. These segments represent portions of the MRT line.

Intersection Check: For each bus route, we check how many of the MRT segments it intersects. This is done by comparing the geometry of the bus route with each MRT segment.

Coverage Calculation: We calculate the percentage of MRT segments that are covered by the bus route:

In [ ]:
# Initialize a list to store the results
segment_coverage_results = []

# Iterate through each row in overlap_df_unique
for idx, row in overlap_df_unique.iterrows():
    # Fetch the geometry of the bus route (LineString) based on ServiceNo and Direction
    bus_geometry = bus_routes_lines.loc[
        (bus_routes_lines['ServiceNo'] == row['Bus_ServiceNo']) &
        (bus_routes_lines['Direction'] == row['Direction'])
    ].geometry.iloc[0]
    
    # Fetch the MRT Line geometry
    mrt_line = line_strings_per_station_line[row['MRT_Line']]['LineString']
    
    # Segment the MRT line 
    mrt_segments = segment_line_by_distance(mrt_line, 1000)  # EDIT

    # Calculate how many segments are covered by the bus route
    total_segments = len(mrt_segments)
    covered_segments = 0

    for segment in mrt_segments:
        if bus_geometry.intersects(segment):  # Check if the bus route intersects the MRT segment
            covered_segments += 1

    # Calculate the percentage of MRT segments covered by the bus route
    coverage_percentage = (covered_segments / total_segments) * 100 if total_segments > 0 else 0
    
    # Append the result to the list
    segment_coverage_results.append({
        'MRT_Line': row['MRT_Line'],
        'Bus_ServiceNo': row['Bus_ServiceNo'],
        'Direction': row['Direction'],
        'Bus_Route_Length_m': row['Bus_Route_Length_m'],
        'Overlap_Length_m': row['Overlap_Length_m'],
        'Coverage_Percentage': coverage_percentage
    })

# Convert the results to a DataFrame
segment_coverage_df = pd.DataFrame(segment_coverage_results)

# Filter the DataFrame to only include rows with a Coverage_Percentage > 0
segment_coverage_df_filtered = segment_coverage_df[segment_coverage_df['Coverage_Percentage'] > 0]

# Sort by coverage percentage in descending order
segment_coverage_df_sorted = segment_coverage_df_filtered.sort_values(by='Coverage_Percentage', ascending=False)

# Output the final DataFrame
segment_coverage_df_sorted

# Phase 2.2 Bus Segments with MRT Line String

The method below segments the bus routes and checks how many consecutive bus segments overlap with the MRT LineString. It also tracks percentage of bus segments that are intersecting with the MRT Linestring in a similar concept to the code above.
This method is in consideration of routes that are not always intersecting with the MRT (like the example given in consult) and hence should not be counted as parallel.

In [ ]:
# add code after fixing bus segments

In [ ]:
# Merge the new bus segments intersection results with the existing coverage DataFrame
phase2_df = pd.merge(
    segment_coverage_df_sorted,  # DataFrame from phase 2.1
    bus_segments_df_sorted,  # DataFrame from phase 2.2 (bus segments)
    on=['MRT_Line', 'Bus_ServiceNo'],  # Merge on common columns
    how='left'  # Keep all rows from phase 2.1 and match with 2.2 results
)

# Output the merged DataFrame
print(phase2_df.head(10))


# Phase 3: Weighted Average Angle of Deviation

This phase aims to assess how parallel bus routes are to MRT segments by calculating the weighted average angle of deviation. The angle is computed between vectors representing each MRT segment and the nearest point on the bus route. Smaller angles indicate closer alignment, suggesting the bus route runs more parallel to the MRT line. The weighted average considers segment lengths, providing a comprehensive measure of overall parallelism. This metric helps to identify potential overlaps and redundancies between bus and MRT services.

In [72]:
# Function to calculate angle between two vectors
def calculate_angle(vector1, vector2):
    # Normalize the vectors to avoid issues with length differences
    vector1 = vector1 / np.linalg.norm(vector1)
    vector2 = vector2 / np.linalg.norm(vector2)

    # Calculate the angle in radians and convert to degrees
    cos_angle = np.dot(vector1, vector2)
    # To prevent possible floating-point inaccuracies that can result in a domain error
    cos_angle = np.clip(cos_angle, -1.0, 1.0)
    angle = np.degrees(np.arccos(cos_angle))
    return angle

In [ ]:
# Suppress the specific warning
warnings.filterwarnings("ignore", message="invalid value encountered in line_locate_point")

# Initialize a list to store the results
segment_coverage_results = []

# Iterate through each row in overlap_df_unique
for idx, row in overlap_df_unique.iterrows():
    # Fetch the geometry of the bus route (LineString) based on ServiceNo and Direction
    bus_geometry = bus_routes_lines.loc[
        (bus_routes_lines['ServiceNo'] == row['Bus_ServiceNo']) &
        (bus_routes_lines['Direction'] == row['Direction'])
    ].geometry.iloc[0]
    
    # Ensure bus geometry is valid
    if not bus_geometry.is_valid:
        bus_geometry = make_valid(bus_geometry)

    # Fetch the MRT Line geometry and segment it
    mrt_line = line_strings_per_station_line[row['MRT_Line']]['LineString']
    if not mrt_line.is_valid:
        mrt_line = make_valid(mrt_line)
    
    mrt_segments = segment_line_by_distance(mrt_line, 1000)

    # Initialize variables to calculate the weighted average angle
    weighted_sum_angle = 0
    total_weight = 0

    # Calculate the angle for each MRT segment and find the closest point on the bus route
    for segment in mrt_segments:
        if not segment.is_valid:
            segment = make_valid(segment)
        
        midpoint = segment.interpolate(0.5, normalized=True)  # Calculate the midpoint of the MRT segment
        try:
            closest_point = bus_geometry.interpolate(bus_geometry.project(midpoint))
        except:
            continue  # Skip this segment if there's an error in projecting
        
        # Create vectors from segment start to end and from midpoint to closest point
        mrt_vector = [segment.coords[-1][0] - segment.coords[0][0], segment.coords[-1][1] - segment.coords[0][1]]
        bus_vector = [closest_point.x - midpoint.x, closest_point.y - midpoint.y]

        # Normalize the vectors
        mrt_vector = np.array(mrt_vector) / np.linalg.norm(mrt_vector)
        bus_vector = np.array(bus_vector) / np.linalg.norm(bus_vector)

        # Calculate angle using dot product (ignoring direction)
        dot_product = np.dot(mrt_vector, bus_vector)
        dot_product = np.clip(dot_product, -1.0, 1.0)  # To avoid numerical errors outside the valid range
        angle = np.degrees(np.arccos(abs(dot_product)))  # Use absolute value to ignore direction

        # Update weighted sum and total weight using segment length 
        segment_length = segment.length
        weighted_sum_angle += angle * segment_length
        total_weight += segment_length

    # Calculate the weighted average angle
    weighted_average_angle = (weighted_sum_angle / total_weight) if total_weight > 0 else 0  
    
    segment_coverage_results.append({
        'MRT_Line': row['MRT_Line'],
        'Bus_ServiceNo': row['Bus_ServiceNo'],
        'Direction': row['Direction'],
        'Weighted_Average_Angle': round(weighted_average_angle, 1)
    })

# Convert to DataFrame and merge with existing data
angles_df = pd.DataFrame(segment_coverage_results)
angles_df_merged = pd.merge(segment_coverage_df_sorted, angles_df, on=['MRT_Line', 'Bus_ServiceNo', 'Direction'], how='left')

angles_df_merged


In [ ]:
# Code to join weighted average angle onto phase 2 final df >> phase 3 df 

# Final Scoring System for Identifying Parallel Routes
In this section, we consolidate all the analyses conducted in previous phases to form a comprehensive scoring system. The final DataFrame (phase3_df) includes critical metrics such as coverage percentage, segment intersections, consecutive coverage, and weighted average angle to determine how well each bus route aligns with the MRT lines.

These metrics are used to systematically evaluate and score parallel routes, allowing us to narrow down on the routes for further evaluation, planning, and studying ridership patterns. The resulting scoring system provides an objective measure for route similarity, facilitating better optimization of public transit services.

In [ ]:
# final scores + rank them
